# Εξαγωγή Μεταδεδομένων Θερμικών Εικόνων
## Χρήση του εργαλείου [exiftool](https://exiftool.org/)
### Παραγωγή csv αρχείων με τα μεταδεδομένα των ετικετών εντός των jpeg 

In [2]:
import os
import sys
import shutil
import yaml
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from celluloid import Camera
from IPython.display import HTML # to show the animation in Jupyter

from skimage.io import imread
from skimage.exposure import histogram
from skimage.filters.rank import entropy
from skimage.morphology import disk, square, skeletonize, thin, dilation
from skimage.util import img_as_ubyte
from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from skimage.measure import label, regionprops, regionprops_table

from sklearn import cluster, preprocessing
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from src.data import data_utils
from src.features import mouse_detection, img_to_vectors
from src.visualization import rect_roi, mask_orientation

# Εισαγωγή του αρχείου yaml
with open(module_path + '/data/multiple_views.yml') as file:
    experiment = yaml.load(file, Loader=yaml.FullLoader)
# access: experiment['samples'], experiment['mouse_id']
dest = module_path + '/data/interim/multiple_metadata'

src.data.data_utils


### Μεταφορά του συστήματος στο directory του `exiftool.exe`

In [ ]:
cd ..

### Κατασκευή των global διευθύνσεων για τα επιλεγμένες εικόνες

In [2]:
infrared_items = []
for i, hour in enumerate(experiment['samples']):
    for mouse_name in experiment['mouse_id']:
        for mouse_angle in experiment['mouse_angle']:
            jpg_file = experiment[mouse_name][mouse_angle]['jpg'][i]
            infrared_items.append(f'{module_path}/data/raw/{hour}/{mouse_name}/{jpg_file}')

### Αντιγραφή των επιλεγμένων δεδομένων και κατασκευή αρχείο μεταδεδομένων

In [ ]:
views_dir = data_utils.make_interim()
views_dir

In [ ]:
for item in infrared_items:
    shutil.copy2(src=item, dst=views_dir)

In [ ]:
!exiftool.exe -csv {views_dir} > {dest}.csv

### Εξαγωγή στατιστικών και συμπερασμάτων από τα μεταδεδομένα

In [3]:
metadata = pd.read_csv(filepath_or_buffer=f'{dest}.csv')

In [3]:
metadata.head()

,SourceFile,ExifToolVersion,FileName,Directory,FileSize,FileModifyDate,FileAccessDate,FileCreateDate,FilePermissions,FileType,FileTypeExtension,MIMEType,JFIFVersion,ExifByteOrder,Make,Model,Orientation,XResolution,YResolution,ResolutionUnit,Software,ModifyDate,YCbCrPositioning,ExposureTime,ExifVersion,CreateDate,ComponentsConfiguration,FocalLength,ImageTemperatureMax,ImageTemperatureMin,FlashpixVersion,ColorSpace,ExifImageWidth,ExifImageHeight,DigitalZoomRatio,ImageUniqueID,Compression,ThumbnailOffset,ThumbnailLength,CreatorSoftware,Emissivity,ObjectDistance,ReflectedApparentTemperature,AtmosphericTemperature,IRWindowTemperature,IRWindowTransmission,RelativeHumidity,PlanckR1,PlanckB,PlanckF,AtmosphericTransAlpha1,AtmosphericTransAlpha2,AtmosphericTransBeta1,AtmosphericTransBeta2,AtmosphericTransX,CameraTemperatureRangeMax,CameraTemperatureRangeMin,CameraTemperatureMaxClip,CameraTemperatureMinClip,CameraTemperatureMaxWarn,CameraTemperatureMinWarn,CameraTemperatureMaxSaturated,CameraTemperatureMinSaturated,CameraModel,CameraPartNumber,CameraSerialNumber,CameraSoftware,LensModel,LensPartNumber,LensSerialNumber,FieldOfView,FilterModel,FilterPartNumber,FilterSerialNumber,PlanckO,PlanckR2,RawValueRangeMin,RawValueRangeMax,RawValueMedian,RawValueRange,DateTimeOriginal,FocusStepCount,FocusDistance,FrameRate,PaletteColors,AboveColor,BelowColor,OverflowColor,UnderflowColor,Isotherm1Color,Isotherm2Color,PaletteMethod,PaletteStretch,PaletteFileName,PaletteName,Palette,RawThermalImageWidth,RawThermalImageHeight,RawThermalImageType,RawThermalImage,DateTimeGenerated,Param0,ImageWidth,ImageHeight,EncodingProcess,BitsPerSample,ColorComponents,YCbCrSubSampling,ImageSize,Megapixels,ShutterSpeed,ThumbnailImage,PeakSpectralSensitivity,FocalLength35efl
0,C:/Users/plouk/Adiposer/data/interim/multiple_...,12.26,IR_2060.jpg,C:/Users/plouk/Adiposer/data/interim/multiple_...,168 KiB,2020:10:20 17:53:22+03:00,2021:06:28 12:17:08+03:00,2021:06:28 12:17:08+03:00,-rw-rw-rw-,JPEG,jpg,image/jpeg,1.01,"Little-endian (Intel, II)",FLIR Systems AB,FLIR E60,Horizontal (normal),72,72,inches,Common_dll v1.6.1b1,2019:04:09 07:42:07,Centered,Jan-59,220,2019:04:09 07:42:07,"-, Cr, Cb, Y",18.0 mm,308,294,100,sRGB,320,240,1,2B7590F0F8230C03D7878F421590B18A,JPEG (old-style),1878,1686,NaN,0.98,0.00 m,20.0 C,22.0 C,20.0 C,1,37.00%,15581.82,1409.8,1,0.006569,0.01262,-0.002276,-0.00667,1.9,120.0 C,-20.0 C,150.0 C,-40.0 C,120.0 C,-20.0 C,150.0 C,-60.0 C,FLIR E60,49001-0602,49031752,20.0.0,FOL18,NaN,NaN,25.0 deg,NaN,NaN,NaN,-6289,0.011337,8135,57222,19100,2760,2019:04:09 07:42:07.394+00:00,2490,0.3 m,30,224,170 128 128,50 128 128,67 216 98,41 110 240,100 128 128,100 110 240,0,2,\FlashFS\system\iron.pal,Iron,"(Binary data 672 bytes, use -b option to extract)",320,240,TIFF,"(Binary data 153804 bytes, use -b option to ex...",2019:04:09 07:42:09,.basicImgData.objectParams.emissivity,320,240,"Baseline DCT, Huffman coding",8,3,YCbCr4:2:0 (2 2),320x240,0.077,Jan-59,"(Binary data 1686 bytes, use -b option to extr...",10.2 um,18.0 mm
1,C:/Users/plouk/Adiposer/data/interim/multiple_...,12.26,IR_2080.jpg,C:/Users/plouk/Adiposer/data/interim/multiple_...,171 KiB,2020:10:20 17:53:22+03:00,2021:06:28 12:17:08+03:00,2021:06:28 12:17:08+03:00,-rw-rw-rw-,JPEG,jpg,image/jpeg,1.01,"Little-endian (Intel, II)",FLIR Systems AB,FLIR E60,Horizontal (normal),72,72,inches,Common_dll v1.6.1b1,2019:04:09 07:45:25,Centered,Jan-59,220,2019:04:09 07:45:25,"-, Cr, Cb, Y",18.0 mm,307,298,100,sRGB,320,240,1,BE6511BB63E2189DC81D5CEB9B77DC34,JPEG (old-style),1878,1682,NaN,0.98,0.00 m,20.0 C,22.0 C,20.0 C,1,37.00%,15581.82,1409.8,1,0.006569,0.01262,-0.002276,-0.00667,1.9,120.0 C,-20.0 C,150.0 C,-40.0 C,120.0 C,-20.0 C,150.0 C,-60.0 C,FLIR E60,49001-0602,49031752,20.0.0,FOL18,NaN,NaN,25.0 deg,NaN,NaN,NaN,-6289,0.011337,8135,57222,19392,1896,2019:04:09 07:45:25.927+00:00,2542,0.3 m,30,224,170 128 128,50 128 128,67 216 98,41 110 240,100 128 128,100 110 240,0,2,\FlashFS\system\iron.pal,Iron,"(Binary data 672 bytes, us

### Tα μεταδεδομένα περιλαμβάνουν 114 χαρακτηριστικά, με τους ακόλουθους τύπους

In [5]:
metadata.info(verbose=True)
# for i, (attribute, dtype) in enumerate(zip(metadata.columns, metadata.dtypes)):
#     print(f'{i}: {attribute}: \t {dtype}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 114 columns):
 #    Column                         Dtype  
---   ------                         -----  
 0    SourceFile                     object 
 1    ExifToolVersion                float64
 2    FileName                       object 
 3    Directory                      object 
 4    FileSize                       object 
 5    FileModifyDate                 object 
 6    FileAccessDate                 object 
 7    FileCreateDate                 object 
 8    FilePermissions                object 
 9    FileType                       object 
 10   FileTypeExtension              object 
 11   MIMEType                       object 
 12   JFIFVersion                    float64
 13   ExifByteOrder                  object 
 14   Make                           object 
 15   Model                          object 
 16   Orientation                    object 
 17   XResolution                    in

### Mετασχηματίζοντας τις ημερομηνίες σε Datetime ISO format

In [6]:
metadata['DateTimeGenerated'] = pd.to_datetime(metadata['DateTimeGenerated'], format="%Y:%m:%d %H:%M:%S")
metadata['DateTimeGenerated']

0     2019-04-09 07:42:09
1     2019-04-09 07:45:32
2     2019-04-09 07:46:16
3     2019-04-09 07:46:51
4     2019-04-09 07:55:25
5     2019-04-09 07:59:49
6     2019-04-09 08:00:07
7     2019-04-09 08:00:27
8     2019-04-09 08:07:33
9     2019-04-09 08:11:24
10    2019-04-09 08:11:51
11    2019-04-09 08:12:56
12    2019-04-09 08:23:14
13    2019-04-09 08:24:07
14    2019-04-09 08:25:28
15    2019-04-09 08:25:54
16    2019-04-09 08:30:51
17    2019-04-09 08:33:05
18    2019-04-09 08:33:33
19    2019-04-09 08:33:55
20    2019-04-10 07:46:01
21    2019-04-10 07:47:58
22    2019-04-10 07:48:30
23    2019-04-10 07:49:01
24    2019-04-10 07:52:55
25    2019-04-10 07:53:40
26    2019-04-10 07:55:50
27    2019-04-10 07:57:16
28    2019-04-10 08:01:03
29    2019-04-10 08:04:34
30    2019-04-10 08:05:26
31    2019-04-10 08:06:23
32    2019-04-10 08:13:57
33    2019-04-10 08:14:49
34    2019-04-10 08:17:37
35    2019-04-10 08:18:05
36    2019-04-10 08:19:17
37    2019-04-10 08:28:22
38    2019-0

In [5]:
description = metadata.describe(include='all', datetime_is_numeric=True)
description

,SourceFile,ExifToolVersion,FileName,Directory,FileSize,FileModifyDate,FileAccessDate,FileCreateDate,FilePermissions,FileType,FileTypeExtension,MIMEType,JFIFVersion,ExifByteOrder,Make,Model,Orientation,XResolution,YResolution,ResolutionUnit,Software,ModifyDate,YCbCrPositioning,ExposureTime,ExifVersion,CreateDate,ComponentsConfiguration,FocalLength,ImageTemperatureMax,ImageTemperatureMin,FlashpixVersion,ColorSpace,ExifImageWidth,ExifImageHeight,DigitalZoomRatio,ImageUniqueID,Compression,ThumbnailOffset,ThumbnailLength,CreatorSoftware,Emissivity,ObjectDistance,ReflectedApparentTemperature,AtmosphericTemperature,IRWindowTemperature,IRWindowTransmission,RelativeHumidity,PlanckR1,PlanckB,PlanckF,AtmosphericTransAlpha1,AtmosphericTransAlpha2,AtmosphericTransBeta1,AtmosphericTransBeta2,AtmosphericTransX,CameraTemperatureRangeMax,CameraTemperatureRangeMin,CameraTemperatureMaxClip,CameraTemperatureMinClip,CameraTemperatureMaxWarn,CameraTemperatureMinWarn,CameraTemperatureMaxSaturated,CameraTemperatureMinSaturated,CameraModel,CameraPartNumber,CameraSerialNumber,CameraSoftware,LensModel,LensPartNumber,LensSerialNumber,FieldOfView,FilterModel,FilterPartNumber,FilterSerialNumber,PlanckO,PlanckR2,RawValueRangeMin,RawValueRangeMax,RawValueMedian,RawValueRange,DateTimeOriginal,FocusStepCount,FocusDistance,FrameRate,PaletteColors,AboveColor,BelowColor,OverflowColor,UnderflowColor,Isotherm1Color,Isotherm2Color,PaletteMethod,PaletteStretch,PaletteFileName,PaletteName,Palette,RawThermalImageWidth,RawThermalImageHeight,RawThermalImageType,RawThermalImage,DateTimeGenerated,Param0,ImageWidth,ImageHeight,EncodingProcess,BitsPerSample,ColorComponents,YCbCrSubSampling,ImageSize,Megapixels,ShutterSpeed,ThumbnailImage,PeakSpectralSensitivity,FocalLength35efl
count,180,1.800000e+02,180,180,180,180,180,180,180,180,180,180,1.800000e+02,180,180,180,180,180.0,180.0,180,180,180,180,180,180.0,180,180,180,180.000000,180.000000,180.0,180,180.0,180.0,180.0,180,180,180.0,180.000000,0.0,1.800000e+02,180,180,180,180,180.0,180,1.800000e+02,1.800000e+02,180.0,1.800000e+02,1.800000e+02,180.000000,1.800000e+02,1.800000e+02,180,180,180,180,180,180,180,180,180,180,180.0,180,180,0.0,0.0,180,0.0,0.0,0.0,180.0,1.800000e+02,180.0,180.0,180.000000,180.000000,180,180.000000,180,180.0,180.0,180,180,180,180,180,180,180.0,180.0,180,180,180,180.0,180.0,180,180,180,180,180.0,180.0,180,180.0,180.0,180,180,1.800000e+02,180,180,180,180
unique,180,NaN,180,1,6,66,3,2,1,1,1,1,NaN,1,1,1,1,NaN,NaN,1,1,180,1,1,NaN,180,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,180,1,NaN,NaN,NaN,NaN,1,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,NaN,2,NaN,NaN,1,1,1,1,1,1,NaN,NaN,1,1,1,NaN,NaN,1,1,180,1,NaN,NaN,1,NaN,NaN,1,1,NaN,1,107,1,1
top,C:/Users/plouk/Adiposer/data/interim/multiple_...,NaN,IR_2529.jpg,C:/Users/plouk/Adiposer/data/interim/multiple_...,170 KiB,2020:10:20 17:53:43+03:00,2021:06:28 12:17:09+03:00,2021:06:28 12:17:09+03:00,-rw-rw-rw-,JPEG,jpg,image/jpeg,NaN,"Little-endian (Intel, II)",FLIR Systems AB,FLIR E60,Horizontal (normal),NaN,NaN,inches,Common_dll v1.6.1b1,2019:04:17 08:45:59,Centered,Jan-59,NaN,2019:04:17 08:45:59,"-, Cr, Cb, Y",18.0 mm,NaN,NaN,NaN,sRGB,NaN,NaN,NaN,2C06F04D96F9720B201197F2719D9526,JPEG (old-style),NaN,NaN,NaN,NaN,0.00 m,20.0 C,22.0 C,20.0 C,NaN,37.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0 C,-20.0 C,150.0 C,-40.0 C,120.0 C,-20.0 C,150.0 C,-60.0 C,FLIR E60,49001-0602,NaN,20.0.0,FOL18,NaN,NaN,25.0 deg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019:04:19 10:36:59.823+00:00,NaN,0.3 m,NaN,NaN,170 128 128,50 128 128,67 216 98,41 110 240,100 128 128,100 110 240,NaN,NaN,\FlashFS\system\iron.pal,Iron,"(Binary data 672 bytes, use -b option to extract)",NaN,NaN,TIFF,"(Binary data 153804 bytes, use -b option to ex...",2019:04:15 08:25:53,.basicImgData.objectParams.emissivity,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,YCbCr4:2:0 (2 2),320x240,NaN,Jan-59,"(Binary data 1697 bytes, use -b option to extr...",10.2 um,18.0 mm
f

In [ ]:
# ποια χαρακτηριστικά επαναλαμβάνονται στα μεταδεδομένα;
if description.loc[count] == 180 and description.loc[unique] == 1:
    print description.loc[top]

### Κατασκευάζοντας τη λίστα με τις  πιο συχνές τιμές (mode)